# Analisis de sensibilidad con gurobi & python


In [1]:
from gurobipy import *

In [2]:
model = Model('model')

# variables
xt = model.addVar(vtype=GRB.CONTINUOUS, name='xt')
xa = model.addVar(vtype=GRB.CONTINUOUS, name='xa')
xg = model.addVar(vtype=GRB.CONTINUOUS, name='xg')
ca = model.addVar(vtype=GRB.CONTINUOUS, name='ca')
va = model.addVar(vtype=GRB.CONTINUOUS, name='va')

# funcion objetivo
model.setObjective(450*xg + 120*xt + 22*va + 28*xa - 28*ca, GRB.MAXIMIZE)

# restricciones
model.addConstr(xt + xa + 0.1*xg <= 800)
model.addConstr(2*xt + 3*xa + 0.05*xg  <= 1000)
model.addConstr(4*xa + ca - 5*xg - va == 0)

# optimizar
model.optimize()

Academic license - for non-commercial use only - expires 2021-05-16
Using license file C:\Users\nesto\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x16cbc942
Coefficient statistics:
  Matrix range     [5e-02, 5e+00]
  Objective range  [2e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02, 1e+03]
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.7400000e+32   4.487500e+30   7.740000e+02      0s
       3    2.4800000e+06   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  2.480000000e+06


In [3]:
# valor funcion objetivo
model.ObjVal

2480000.0

In [4]:
# Valor de las variables
model.getVars()

[<gurobi.Var xt (value 0.0)>,
 <gurobi.Var xa (value 0.0)>,
 <gurobi.Var xg (value 8000.0)>,
 <gurobi.Var ca (value 40000.0)>,
 <gurobi.Var va (value 0.0)>]

In [5]:
xg.x

8000.0

In [6]:
print("valor Funcion Objetivo: "+ str(model.ObjVal))
for v in model.getVars():
    print(str(v.varName)+" "+str(v.x))

valor Funcion Objetivo: 2480000.0
xt 0.0
xa 0.0
xg 8000.0
ca 40000.0
va 0.0


In [7]:
model.status

2

In [8]:
# Nombre de la variable
xt.varName
# Costo reducido
xt.RC

-2980.0

In [9]:
# Costos reducidos
for v in model.getVars():
    print(str(v.varName)+" Costo reducido: "+str(v.RC))

xt Costo reducido: -2980.0
xa Costo reducido: -2960.0
xg Costo reducido: 0.0
ca Costo reducido: 0.0
va Costo reducido: -6.0


In [10]:
# Holguras:

# Paso 1: Obtener ;as restricciones del modelo
model.getConstrs()

[<gurobi.Constr R0>, <gurobi.Constr R1>, <gurobi.Constr R2>]

In [20]:
# Obtener las holguras de cada restriccion
for r in model.getConstrs():
    print(str(r)[-3:-1]+" holgura: "+str(r.slack))

R0 holgura: 0.0
R1 holgura: 600.0
R2 holgura: 0.0


In [21]:
# Precios Duales
for r in model.getConstrs():
    print(str(r)[-3:-1]+" Precio Dual: "+str(r.pi))





R0 Precio Dual: 3100.0
R1 Precio Dual: 0.0
R2 Precio Dual: -28.0


In [27]:
# Analisis de sensibilidad de los coeficientes
xt.SAObjup
for v in model.getVars():
    print(str(v.varName)+" Sensibilidad Inferior: "+str(v.SAObjLow) + "| superior :"+ str(v.SAObjUp))

xt Sensibilidad Inferior: -inf| superior :3100.0
xa Sensibilidad Inferior: -inf| superior :2988.0
xg Sensibilidad Inferior: 154.0| superior :inf
ca Sensibilidad Inferior: -82.81481481481481| superior :-22.0
va Sensibilidad Inferior: -inf| superior :28.0


In [29]:
# Sensibilidad del lado derecho
for r in model.getConstrs():
    print(str(r)[-3:-1]+" la derecho: "+str(r.RHS) + " intervalos: "+ str(r.SARHSLow)+' '+ str(r.SARHSUp) )


R0 la derecho: 800.0 intervalos: 0.0 2000.0
R1 la derecho: 1000.0 intervalos: 400.0 inf
R2 la derecho: 0.0 intervalos: -40000.0 inf
